# 4일차 3교시 데이터 타입

### 목차
* 1. 개요
* 2. 스파크 데이터 타입으로 변환하기
* 3. 불리언 데이터 타입 다루기
* 4. 수치형 데이터 타입 다루기
* 5. 문자열 데이터 타입 다루기
* 6. 정규표현식
* 7. 날짜와 타임스팸프 데이터 타입 다루기
* 8. 널 값 다루기
* 9. 복합 데이터 다루기
* 10. JSON 다루기
* 11. 사용자 정의 함수 


### 1. 개요
+ DataFrame 메서드
    + DataFrameStatFunctions : 다양한 통계적 함수를 제공
    + DataFrameNaFunctions : null 데이터를 다루는데 필요한 함수를 제공
+ Column 메서드
    + alias, contains과 같은 컬럼과 관련된 여러가지 메서드를 제공
    + org.apache.spark.sql.function 패키지는 데이터 타입과 관련된 다양한 함수를 제공

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Data Engineer Intermediate Day4") \
    .config("spark.dataengineer.intermediate.day4", "tutorial-3") \
    .getOrCreate()

spark.sparkContext.getConf().getAll() 

[('spark.app.id', 'local-1598173575671'),
 ('spark.driver.host', '7123e3456028'),
 ('spark.driver.port', '43995'),
 ('spark.app.name', 'Data Engineer Intermediate Day4'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.dataengineer.intermediate.day4', 'tutorial-3'),
 ('spark.ui.showConsoleProgress', 'true')]

In [3]:
""" DataFrame 생성 """
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("retail")
df.show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365| 

### 2. 스파크 데이터 타입으로 변환하기
+ lit 함수 : 상수 값을 리터럴 데이터 타입으로 변환합니다
+ spark SQL에서는 스파크 데이터 타입으로 변경할 수 없음

In [4]:
""" 스파크 데이터 타입으로 변환 """
from pyspark.sql.functions import lit

df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

### 3. 불리언 데이터 타입 다루기

In [17]:
""" 조건절 입력 """
from pyspark.sql.functions import col

x1 = df.where(col("InvoiceNO") != 536365).select("InvoiceNO", "Description")
x2 = df.where("InvoiceNO <> 536365").select("InvoiceNO", "Description")
x3 = df.where("InvoiceNO = 536365").select("InvoiceNO", "Description")

# 아래는 집합 연산을 통해 subtract 연산을 할 수 있습니다
assert(0 == x1.subtract(x2).count())
assert(0 == x2.subtract(x1).count())
x1.show(5)
x2.show(5)

+---------+--------------------+
|InvoiceNO|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
|   536367|ASSORTED COLOUR B...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|POPPY'S PLAYHOUSE...|
+---------+--------------------+
only showing top 5 rows

+---------+--------------------+
|InvoiceNO|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
|   536367|ASSORTED COLOUR B...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|POPPY'S PLAYHOUSE...|
+---------+--------------------+
only showing top 5 rows



In [5]:
""" an, or 사용한 불리언 표현식 """

from pyspark.sql.functions import instr

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1 # Locate the position of the first occurrence of substr column in the given string.
                                                      # Returns null if either of the arguments are null.
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [30]:
# SparkSQL 을 이용한 is in 구문 사용
from pyspark.sql.functions import desc
df.select('StockCode').where("StockCode in ('DOT', 'POST', 'C2')").distinct().show(5)
df.select('StockCode').distinct().sort(desc('StockCode')).show(5)

+---------+
|StockCode|
+---------+
|      DOT|
|     POST|
|       C2|
+---------+

+---------+
|StockCode|
+---------+
|     POST|
|        M|
|      DOT|
|        D|
|       C2|
+---------+
only showing top 5 rows



In [32]:
from pyspark.sql.functions import *
""" instr 함수 """
df.withColumn("added", instr(df.Description, "POSTAGE")).where("added > 1").show() # 8번째 글자에 'POSTAGE'가 시작됨

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|added|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|    8|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|    8|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+



In [33]:
""" 불리언 컬럼 """
from pyspark.sql.functions import instr

DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") > 1 # 'POSTAGE' 글자가 첫번째가 아닌 경우
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter)) \
    .where("isExpensive") \
    .select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



In [34]:
""" SQL문 실행 """
from pyspark.sql.functions import expr, col # 파이썬은 not이 존재하지 않는다. 

df.withColumn("isExpensive", expr("NOT UnitPrice <= 250")) \
    .where("isExpensive") \
    .select("Description", "UnitPrice").show(5)
# SQL을 사용해도 성능은 차이나지 않는다.

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



### 4. 수치형 데이터 타입 다루기

In [35]:
""" 지수만큼 제곱하는 pow 함수 """
from pyspark.sql.functions import expr, pow

# 아래의 연산이 필요한 경우에는 반드시 column 으로 지정되어야 연산자 계산이 됩니다. (문자열 * 연산자는 없습니다)
fabricateQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerID"), fabricateQuantity.alias("realQuantity")).show(5)

+----------+------------------+
|CustomerID|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 5 rows



In [38]:
# 구조화 API 가 어렵다면 Expression 을 이용하여 표현해도 결과는 동일합니다.
fabricateQuantity = expr("pow(Quantity * UnitPrice, 2) + 5")
df.select(expr("CustomerID"), fabricateQuantity.alias("realQuantity")).show(5)

+----------+------------------+
|CustomerID|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 5 rows



In [40]:
""" 반올림하는 round 함수 """
from pyspark.sql.functions import lit, round, bround # round(반올림), bround(반내림)

df.select(round(lit("2.5"), 1), bround(lit("2.5"), 1)).show(2) # 1차원

+-------------+--------------+
|round(2.5, 1)|bround(2.5, 1)|
+-------------+--------------+
|          2.5|           2.5|
|          2.5|           2.5|
+-------------+--------------+
only showing top 2 rows



In [41]:
""" 피어슨 상관계수를 계산 """
from pyspark.sql.functions import corr

df.stat.corr("Quantity", "UnitPrice")

-0.04112314436835551

In [42]:
""" 요약 통계를 계산 """
df.describe().show()
df.describe("InvoiceNo").show() # 컬럼을 입력

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|               0.0|           12431.0|     Australia|
|    max|          C

+ StaFunctions 패키지의 통계 함수

In [43]:
""" 백분위수를 구하는 approxQuantile """
olName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05

df.stat.approxQuantile("UnitPrice", quantileProbs, relError)
# :relError: The relative target precision to achieve
#   (>= 0). If set to zero, the exact quantiles are computed, which
#   could be very expensive. Note that values greater than 1 are
#   accepted but give the same result as 1.

[2.51]

In [48]:
""" 교차표를 생성하는 crosstab """
df.select("StockCode").distinct().show(5) # rows name
df.select("Quantity").distinct().show(5)  # columns name
df.stat.crosstab("StockCode", "Quantity").show(5) # pivot

+---------+
|StockCode|
+---------+
|    22728|
|    21889|
|   90210B|
|    21259|
|    21894|
+---------+
only showing top 5 rows

+--------+
|Quantity|
+--------+
|      34|
|      -1|
|      28|
|      27|
|     384|
+--------+
only showing top 5 rows

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-

In [50]:
""" 고유 아이디를 생성하는 monotonically_increasing_id """
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show(5)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
+-----------------------------+
only showing top 5 rows



### 5. 문자열 데이터 타입 다루기

In [51]:
""" 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변경, initcap """ 
from pyspark.sql.functions import initcap

df.select(initcap(col("Description"))).show(2, False)

+----------------------------------+
|initcap(Description)              |
+----------------------------------+
|White Hanging Heart T-light Holder|
|White Metal Lantern               |
+----------------------------------+
only showing top 2 rows



In [52]:
""" 소문자/대문자 변경, lower/upper """
from pyspark.sql.functions import lower, upper

df.select(col("Description"), lower(col("Description")), upper(col("Description"))).show(2)

+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN|
+--------------------+--------------------+--------------------+
only showing top 2 rows



In [53]:
""" 문자열 주변의 공백을 제거, lpad/ltrim/rpad/rtrim/trim """
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim

df.select(
    ltrim(lit("   HELLO   ")).alias("ltrim"),
    rtrim(lit("   HELLO   ")).alias("rtrim"),
    trim(lit("   HELLO   ")).alias("trim"),
    lpad(lit("HELLO"), 3, " ").alias("lp"),
    rpad(lit("HELLO"), 10, " ").alias("rp")
).show(2)

+--------+--------+-----+---+----------+
|   ltrim|   rtrim| trim| lp|        rp|
+--------+--------+-----+---+----------+
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
+--------+--------+-----+---+----------+
only showing top 2 rows



### 6. 정규 표현식
+ 존재 여부를 확인하거나 일치하는 모든 문자열을 치환
+ 정규 표현식을 위해 regexp_extract 함수와 regexp_replace 함수를 제공

In [54]:
""" 단어 치환, regexp_extract """
from pyspark.sql.functions import regexp_replace

regex_string = "BLACK|WHITE|RED|GRENN|BLUE"
df.select(
    regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
    col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [59]:
""" 문자 치환, translate """
from pyspark.sql.functions import translate
df.select(
    col("Description"),
    translate(col("Description"), "LEET", "12").alias("Translated") # 정확히 매칭되지 않아도 부분만 적용됩니다 L:1, E:2
).show(5)

+--------------------+--------------------+
|         Description|          Translated|
+--------------------+--------------------+
|WHITE HANGING HEA...|WHI2 HANGING H2AR...|
| WHITE METAL LANTERN|    WHI2 M2A1 1AN2RN|
|CREAM CUPID HEART...|CR2AM CUPID H2ARS...|
|KNITTED UNION FLA...|KNI2D UNION F1AG ...|
|RED WOOLLY HOTTIE...|R2D WOO11Y HOI2 W...|
+--------------------+--------------------+
only showing top 5 rows



In [60]:
""" 단어 추출, regexp_extract """
from pyspark.sql.functions import regexp_extract

extract_str = "(BLACK|WHITE|RED|GRENN|BLUE)"
df.select(
    col("Description"),
    regexp_extract(col("Description"), extract_str, 1).alias("Extracted")
).show(2)

+--------------------+---------+
|         Description|Extracted|
+--------------------+---------+
|WHITE HANGING HEA...|    WHITE|
| WHITE METAL LANTERN|    WHITE|
+--------------------+---------+
only showing top 2 rows



In [61]:
""" 단어 존재유무, contain """ # 파이썬과 SQL은 instr 함수를 사용
from pyspark.sql.functions import instr

containBlack = instr(col("Description"), "BLACK") > 1
containWhite = instr(col("Description"), "WHITE") > 1
df.withColumn("hasSimpleColor", containBlack | containWhite) \
    .where("hasSimpleColor") \
    .select("Description") \
    .show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|RED WOOLLY HOTTIE WHITE HEART.    |
|WOOD 2 DRAWER CABINET WHITE FINISH|
|WOOD S/3 CABINET ANT WHITE FINISH |
+----------------------------------+
only showing top 3 rows



In [62]:
""" 필드에 색깔 문자열이 포함되어 있는지 여부를 locate 함수를 이용하여 컬럼으로 생성하는 에제 """
from pyspark.sql.functions import expr, locate 

simple_colors = ["black", "white", "red", "green", "blue"]

def color_locator(column, color_string):   # color_strings 단어가 시작되는 문자기준(단어기준 X) 위치
    return locate(color_string.upper(), column).cast("boolean").alias("is_" + color_string)

selected_cols = [color_locator(df.Description, c) for c in simple_colors] # locate 함수를 하나씩 list에 저장
selected_cols.append(expr("*")) # column 타입이여야 함 

df.select(*selected_cols).show(3)

df.select(*selected_cols).where(expr("is_white OR is_red")) \
    .select(col("Description")) \
    .show(3, False)

+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|is_black|is_white|is_red|is_green|is_blue|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   false|    true| false|   false|  false|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   false|    true| false|   false|  false|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   false|   false| false|   false|  false|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------

In [63]:
selected_cols

[Column<b'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`'>,
 Column<b'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`'>,
 Column<b'CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`'>,
 Column<b'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`'>,
 Column<b'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`'>,
 Column<b'unresolvedstar()'>]

### 7. 날짜와 타임스팸프 데이터 타입 다루기
+ 스파크는 2가지 시간 정보만 다룸
    + 날짜 정보만 가지는 date
    + 날짜와 시간 정보를 모두 가지는 timestamp
+ 시간대 설정이 필요하다면 스파크 SQL 설정의 spark.conf.sessionLocalTimeZone 속성으로 가능
    + 자바 TimeZone 포맷을 따라야 함
+ TimestampType 클래스는 초 단위 정밀도만 지원
    + 초 단위 이상 정밀도 요구 시 long 데이터 타입으로 데이터를 변환해 처리하는 우회 정책이 필요

In [68]:
""" 오늘 날짜 구하기 """
from pyspark.sql.functions import current_date, current_timestamp

dateDF = spark.range(10) \
    .withColumn("today", current_date()) \
    .withColumn("now", current_timestamp())

dateDF.createOrReplaceTempView("dataTable")
dateDF.printSchema()

dateDF.show(3, False)

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2020-08-23|2020-08-23 09:44:23.097|
|1  |2020-08-23|2020-08-23 09:44:23.097|
|2  |2020-08-23|2020-08-23 09:44:23.097|
+---+----------+-----------------------+
only showing top 3 rows



In [69]:
""" 날짜를 더하거나 빼기 """
from pyspark.sql.functions import date_sub, date_add

dateDF.select(
    date_sub(col("today"), 5),
    date_add(col("today"), 5)
).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2020-08-18|        2020-08-28|
+------------------+------------------+
only showing top 1 row



In [70]:
""" 두 날짜 사이의 일/개월 수를 파악 """
from pyspark.sql.functions import datediff, months_between, to_date

dateDF.withColumn("week_ago", date_sub(col("today"), 7)) \
    .select(datediff(col("week_ago"), col("today"))) \
    .show(1) # 현재 날짜에서 7일 제외 후 datediff 결과 확인

dateDF \
    .select(to_date(lit("2016-01-01")).alias("start"), to_date(lit("2017-05-22")).alias("end")) \
    .select(months_between(col("start"), col("end"))).show(1) # 개월 수 차이 파악

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [71]:
""" 문자열을 날짜로 변환 """ # 자바의 simpleDateFormat 클래스가 지원하는 포맷 사용 필요
from pyspark.sql.functions import to_date, lit

spark.range(5) \
    .withColumn("date", lit("2017-01-01")) \
    .select(to_date(col("date"))) \
    .show(1)

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
+---------------+
only showing top 1 row



In [72]:
""" 파싱오류로 날짜가 null로 반환되는 사례 """
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show(1) # 월과 일의 순서가 바뀜

+---------------------+---------------------+
|to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
|                 null|           2017-12-11|
+---------------------+---------------------+
only showing top 1 row



In [73]:
""" SimpleDateFormat 표준을 활용하여 날짜 포멧을 지정 """
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM" # 소문자 mm 주의
cleanDateDF = spark.range(1).select( # 1개 Row를 생성
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

※ SimpleDateFormat : https://bvc12.tistory.com/168

In [74]:
spark.sql("""SELECT * from dateTable2""").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [75]:
""" 항상 날짜 포맷을 지정해야 하는 to_timestamp 함수 """
from pyspark.sql.functions import to_timestamp

cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+----------------------------------+
|to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
|               2017-11-12 00:00:00|
+----------------------------------+



In [76]:
""" 날짜 비교 """
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



### 8. 널 값 다루기
+ null 값을 사용하는 것 보다 명시적으로 사용하는 것이 항상 좋음
+ null 값을 허용하지 않는 컬럼을 선언해도 강제성은 없음
+ nullable 속성은 스파크 SQL 옵티마이저가 해당 컬럼을 제어하는 동작을 단순하게 돕는 역할
+ null 값을 다루는 방법은 두 가지 
    + 명시적으로 null을 제거
    + 전역 또느 컬럼 단위로 null 값을 특정 값으로 채움

### 8-1. coalesce w/ null

In [77]:
# coalesce : 인수로 지정한 여러 컬럼 중 null이 아닌 첫번 째 값 반환 하는 함수
from pyspark.sql.functions import coalesce 

df.select(coalesce(col("Description"), col("CustomerId"))).show()

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



### 8-2. ifnull, nullIf, nvl, nvl2
+ SQL 함수이며 DataFrame의 select 표현식으로 사용 가능
    + ifnull(null, 'return_value') # 두 번째 값을, 아니라면 첫 번째 값을 반환 
    + nullif('value', 'value')     # 두 값이 같으면 null
    + nvl(null, 'return_value')    # 두 번째 값을, 아니라면 첫 번째 값을 반환
    + nvl2('not_null', 'return_value', 'else_value') # 두 번째 값을, 아니라면 세번째 값을 반환

In [79]:
spark.sql("""
SELECT
    ifnull(null, 'return_value'),
    nullif('value', 'value'),
    nvl(null, 'return_value'),
    nvl2('not null', 'return_value', 'else_value')
""").show()

+----------------------------+------------------------+-------------------------+----------------------------------------------+
|ifnull(NULL, 'return_value')|nullif('value', 'value')|nvl(NULL, 'return_value')|nvl2('not null', 'return_value', 'else_value')|
+----------------------------+------------------------+-------------------------+----------------------------------------------+
|                return_value|                    null|             return_value|                                  return_value|
+----------------------------+------------------------+-------------------------+----------------------------------------------+



### 8-3 drop
+ null 값을 가진 로우를 제거

In [80]:
df.na.drop()
df.na.drop("any").show(1) # 로우 컬럼값 중 하나라도 null이면 제거
df.na.drop("all").show(1) # 로우 컬럼값 모두 null이면 제거

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---

In [81]:
# 배열 형태의 컬럼을 인수로 전달하여 지정한 컬럼만 제거합니다
df.na.drop("all", subset=("StockCode", "InvoiceNo")).show(1)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row



### 8.4 fill
+ fill: null을 특정한 값으로 채움

In [82]:
""" null을 포함한 DataFrame 행성 """
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, DoubleType

myManualSchema = StructType([
    StructField("string_null", StringType(), True),
    StructField("string2_null", StringType(), True),
    StructField("number_null", DoubleType(), True)
])

myRows = []
myRows.append(Row("Hello", None, float(5))) # string 컬럼에 null 포함
myRows.append(Row(None, "World", None))     # number 컬럼에 null 포함

myDf = spark.createDataFrame(myRows, myManualSchema)
myDf.show()

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|        null|        5.0|
|       null|       World|       null|
+-----------+------------+-----------+



+ fill 함수는 DataType이 동일한 컬럼의 null만 치완
+ 숫자형 또한 치환할 값의 DataType이 동일해야 함

In [83]:
myDf.na.fill("All null valus become this string").show()
myDf.na.fill(5.0).show() 

+--------------------+--------------------+-----------+
|         string_null|        string2_null|number_null|
+--------------------+--------------------+-----------+
|               Hello|All null valus be...|        5.0|
|All null valus be...|               World|       null|
+--------------------+--------------------+-----------+

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|        null|        5.0|
|       null|       World|        5.0|
+-----------+------------+-----------+



In [84]:
""" 딕셔너리 타입을 사용해서 다수의 컬럼에 fill 메서드를 적용 """
fill_cols_vals = {"number_null": 5.0, "string_null": "No Value"}
myDf.na.fill(fill_cols_vals).show()

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|        null|        5.0|
|   No Value|       World|        5.0|
+-----------+------------+-----------+



### 8-5 replace

In [85]:
""" 조건에 따라 다른 값으로 대체 """
myDf.na.replace([""], ["Hello"], "string_null").show() # null을 지정하는 방법은?

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|        null|        5.0|
|       null|       World|       null|
+-----------+------------+-----------+



## 8-6 정렬하기
> asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 참조

### 9. 복합 데이터 다루기
+ 구조체, 배열, 맵

#### 9-1 구조체
+ DataFrame 내부의 DataFrame
+ 다수의 컬럼을 괄호로 묶어 생성 가능
+ 문법에 점(.)을 사용하거나 getField 메서드를 사용
+ (*) 문자로 모든 값을 조회할 수 있음

In [87]:
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
complexDF.show(5, False)
complexDF.printSchema()

+---------------------------------------------+
|complex                                      |
+---------------------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER, 536365] |
|[WHITE METAL LANTERN, 536365]                |
|[CREAM CUPID HEARTS COAT HANGER, 536365]     |
|[KNITTED UNION FLAG HOT WATER BOTTLE, 536365]|
|[RED WOOLLY HOTTIE WHITE HEART., 536365]     |
+---------------------------------------------+
only showing top 5 rows

root
 |-- complex: struct (nullable = false)
 |    |-- Description: string (nullable = true)
 |    |-- InvoiceNo: string (nullable = true)



In [44]:
complexDF.select("complex.Description", "complex.InvoiceNo") # 모두 동일
complexDF.select(col("complex").getField("Description"), col("complex").getField("InvoiceNo"))
complexDF.select("complex.*")
complexDF.select(col("complex.*"))
complexDF.selectExpr("complex.*").show(5)

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
|CREAM CUPID HEART...|   536365|
|KNITTED UNION FLA...|   536365|
|RED WOOLLY HOTTIE...|   536365|
+--------------------+---------+
only showing top 5 rows



#### 9-2 배열
+ 데이터에서 Description 컬럼의 모든 단어를 하나의 로우로 변환

#### Split

In [88]:
""" 컬럼을 배열로 변환 """
from pyspark.sql.functions import split

df.select(split(col("Description"), " ")).show(2)

+---------------------+
|split(Description,  )|
+---------------------+
| [WHITE, HANGING, ...|
| [WHITE, METAL, LA...|
+---------------------+
only showing top 2 rows



In [89]:
""" 배열값의 조회 """
df.select(split(col("Description"), " ").alias("array_col"))\
    .selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



#### 배열의 길이

In [90]:
""" size 함수 """
from pyspark.sql.functions import size

df.select(size(split(col("Description"), " "))).show(2)

+---------------------------+
|size(split(Description,  ))|
+---------------------------+
|                          5|
|                          3|
+---------------------------+
only showing top 2 rows



#### array_contains
+ array_contains 함수를 사용해 배열에 특정 값이 존재하는지 확인

In [91]:
from pyspark.sql.functions import array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

+--------------------------------------------+
|array_contains(split(Description,  ), WHITE)|
+--------------------------------------------+
|                                        true|
|                                        true|
+--------------------------------------------+
only showing top 2 rows



#### explode
+ 배열 타입의 컬럼을 입력받고 컬럼의 배열값에 포함된 모든 값을 로우로 변환

In [96]:
from pyspark.sql.functions import split, explode
exploded = df \
    .withColumn("splitted", split(col("Description"), " ")) \
    .withColumn("exploded", explode(col("splitted")))
exploded.printSchema()

ef = exploded.select("Description", "InvoiceNo", "exploded") # 모든 단어가 하나의 로우로 전환됨
print(df.select("Description").count())
print(ef.select("exploded").count()) # 로우 수가 다름

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- splitted: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- exploded: string (nullable = true)

3108
14414


In [97]:
explodedDf.select("Description", "exploded").count() # 큰 쪽으로 카운드

14414

In [98]:
explodedDf.select("Description", "exploded").take(10) # Description 컬럼이 Group이 되어 중복됨

[Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='WHITE'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HANGING'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HEART'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='T-LIGHT'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HOLDER'),
 Row(Description='WHITE METAL LANTERN', exploded='WHITE'),
 Row(Description='WHITE METAL LANTERN', exploded='METAL'),
 Row(Description='WHITE METAL LANTERN', exploded='LANTERN'),
 Row(Description='CREAM CUPID HEARTS COAT HANGER', exploded='CREAM'),
 Row(Description='CREAM CUPID HEARTS COAT HANGER', exploded='CUPID')]

#### 9-3 맵
+ map 함수와 컬럼의 키0값 쌍을 이용해 생성
+ 적합한 키를 사용해 데이터를 조회할 수 있으며, 해당키가 없다면 null값을 반환

In [112]:
""" 맵 생성 """
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).show(20, False)

+-----------------------------------------------+
|complex_map                                    |
+-----------------------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER -> 536365] |
|[WHITE METAL LANTERN -> 536365]                |
|[CREAM CUPID HEARTS COAT HANGER -> 536365]     |
|[KNITTED UNION FLAG HOT WATER BOTTLE -> 536365]|
|[RED WOOLLY HOTTIE WHITE HEART. -> 536365]     |
|[SET 7 BABUSHKA NESTING BOXES -> 536365]       |
|[GLASS STAR FROSTED T-LIGHT HOLDER -> 536365]  |
|[HAND WARMER UNION JACK -> 536366]             |
|[HAND WARMER RED POLKA DOT -> 536366]          |
|[ASSORTED COLOUR BIRD ORNAMENT -> 536367]      |
|[POPPY'S PLAYHOUSE BEDROOM  -> 536367]         |
|[POPPY'S PLAYHOUSE KITCHEN -> 536367]          |
|[FELTCRAFT PRINCESS CHARLOTTE DOLL -> 536367]  |
|[IVORY KNITTED MUG COSY  -> 536367]            |
|[BOX OF 6 ASSORTED COLOUR TEASPOONS -> 536367] |
|[BOX OF VINTAGE JIGSAW BLOCKS  -> 536367]      |
|[BOX OF VINTAGE ALPHABET BLOCKS -> 536367]     |


In [108]:
""" 맵의 데이터 조회 """
mapped = df \
    .select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))
mapped.printSchema()
mapped.selectExpr("complex_map['WHITE METAL LANTERN']").where("complex_map['WHITE METAL LANTERN'] is not null").show()

root
 |-- complex_map: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                          536365|
|                          536373|
|                          536375|
|                          536396|
|                          536406|
|                          536544|
+--------------------------------+



In [114]:
""" 맵의 분해 """
exploded = df \
    .select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")) \
    .selectExpr("explode(complex_map)")
exploded.printSchema()
exploded.show(5)

root
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
|CREAM CUPID HEART...|536365|
|KNITTED UNION FLA...|536365|
|RED WOOLLY HOTTIE...|536365|
+--------------------+------+
only showing top 5 rows



### 10. JSON 다루기

In [115]:
""" Json 컬럼 생성 """
jsonDF = spark.range(1).selectExpr(
    """
    '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString
    """
)

In [116]:
""" 인라인 쿼리로 JSON 조회하기 """
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
    get_json_object(col("jsonString"), "jsonString.myJSONKey.myJSONValue[1]").alias("column"),
    json_tuple(col("jsonString"), "myJSONKey")
).show(2)

+------+--------------------+
|column|                  c0|
+------+--------------------+
|  null|{"myJSONValue":[1...|
+------+--------------------+



In [119]:
""" StructType을 Json 문자열로 변경 """
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as myStruct") \
    .select(to_json(col("myStruct"))) \
    .take(3)

[Row(structstojson(myStruct)='{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}'),
 Row(structstojson(myStruct)='{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}'),
 Row(structstojson(myStruct)='{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}')]

In [120]:
""" Json 문자열을 객체로 변환 """
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

parseSchema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("Description", StringType(), True)
])

df.selectExpr("(InvoiceNo, Description) as myStruct") \
    .select(to_json(col("myStruct")).alias("newJSON")) \
    .select(from_json(col("newJSON"), parseSchema), col("newJSON")) \
    .show(2) # 키를 컬럼명으로 값을 로우로 변경

+----------------------+--------------------+
|jsontostructs(newJSON)|             newJSON|
+----------------------+--------------------+
|  [536365, WHITE HA...|{"InvoiceNo":"536...|
|  [536365, WHITE ME...|{"InvoiceNo":"536...|
+----------------------+--------------------+
only showing top 2 rows



### 11. 사용자 정의 함수 
+ User defined function(UDF)는 레포트별로 데이터를 처리하는 함수이며, SparkSession이나 Context에서 사용할 수 있도록 임시 함수 형태로 등록됨
+ 내장 함수가 제공하는 코드 생성 기능의 장점을 활용할 수 없어 약간의 성능 저하 발생
+ 언어별로 성능차이가 존재, 파이썬에서도 사용할 수 있으므로 자바나 스칼라도 함수 작성을 추천

In [121]:
""" UDF 사용하기 """
udfExDF = spark.range(5).toDF("num")
def power3(double_value):
    return double_value ** 3
power3(2.0)

8.0

In [122]:
""" UDF 등록 및 사용 """
from pyspark.sql.functions import udf
power3udf = udf(power3)

udfExDF.select(power3udf(col("num"))).show(2)

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
+-----------+
only showing top 2 rows



스칼라에서 등록되 사용자 정의 함수를 파이썬에서 활용:

https://www.cyanny.com/2017/09/15/spark-use-scala-udf-udaf-in-pyspark/